In [12]:
from dotenv import load_dotenv
from langchain_fireworks import ChatFireworks
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

load_dotenv()
llm = ChatFireworks(model="accounts/fireworks/models/llama-v3-70b-instruct",temperature=0)
system_message_prompt = SystemMessagePromptTemplate.from_template("你是一个求职助手，用汉语交流。")
human_message_prompt = HumanMessagePromptTemplate.from_template("HR问或说：“{user_input}”，你用汉语回答：")
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt,
     human_message_prompt]
)
chain = chat_prompt|llm|StrOutputParser()
text = """
请问，最近有换工作的意愿么？我们正在寻找一位团队伙伴。
"""
messages = chat_prompt.format_prompt(user_input=text)
print(chain.invoke(messages))



"嗨，感谢您的邀请！我目前确实有换工作的意愿，想寻找一个挑战性的角色和良好的团队文化。您的团队是什么样的？有什么样的项目和目标？"


In [20]:
from langchain_fireworks import ChatFireworks
from dotenv import load_dotenv
load_dotenv()
llm = ChatFireworks(model="accounts/fireworks/models/llama-v3-70b-instruct",temperature=0)
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate
)
from langchain_core.output_parsers import StrOutputParser

# 示例样本 给出模板中使用的变量的值
examples = [
    {"text": "离职/换工作的原因","label": "离职原因"},
    {"text": "你好，我们是外协岗位，在国家电网 南瑞工作的","label": "外包&外协&外派&驻场"},
    {"text": "但是我们应该最高30K，一般还达不到.","label": "薪资"},
    {"text": "哈喽～本职位为线上兼职，一单一结款，根据自己时间自由接单，不耽误自己的主业，您看感兴趣嘛？","label":"兼职"}
]
# 示例样本的模板 给出示例的具体格式
example_prompt = PromptTemplate.from_template(
"""文本: {text}
类别: {label}
"""
)
# 放在所有示例之前的文字，用于描述任务
prefix = """
给出每个文本的类别，类别只能属于以下列出的一种

- 离职原因
- 薪资
- 外包&外协&外派&驻场
- 兼职
- 学历

如果不属于以上类别，则类别名称为“其他”。

例如：
"""
# 放在所有示例之后的文字，用于输入用户文本让LLM预测
suffix = """文本: {input}\n类别:
"""
# 带有示例的提示模板
few_shot_prompt = FewShotPromptTemplate(
    examples=examples, # 示例列表
    example_prompt=example_prompt, # 示例提示
    prefix=prefix, # 放在所有示例之前的文字，用于描述任务
    suffix=suffix, # 放在所有示例之后的文字，用于输入用户文本让LLM预测
    input_variables=["input"], # 用户的输入变量名列表
    example_separator="\n" # 示例之间的分隔符
)
# 接收用户输入变量的值生成少样本提示
# print(few_shot_prompt.format(input='你好'))
chain = few_shot_prompt|llm|StrOutputParser()
import re
text_li = [
    "你好，这边是兼职，可在家/空闲时间接单。爬虫，代码实现/跑通，数据分析，图像算法等等。",
    "你好，我们是软通动力正在以20-40K的薪资招聘大模型算法工程师(线上面试+周末双休)，看了你的简历，觉得与我们的岗位要求非常匹配～盼进一步沟通～",
    "于先生，可以就你的经历详细说说么？",
    "您好，这个岗位是外派驻场性质的哦",
    "你好，我们正在诚聘nlp算法（外包银行），有兴趣聊聊吗",
    "离职状态吗？",
]

for text in text_li:
    text = text.strip()
    result = chain.invoke({"input":text})
    result = re.sub('类别: ?','',result)
    print(f"文本: {text}\n类别: {result}")

文本: 你好，这边是兼职，可在家/空闲时间接单。爬虫，代码实现/跑通，数据分析，图像算法等等。
类别: 兼职
文本: 你好，我们是软通动力正在以20-40K的薪资招聘大模型算法工程师(线上面试+周末双休)，看了你的简历，觉得与我们的岗位要求非常匹配～盼进一步沟通～
类别: 薪资
文本: 于先生，可以就你的经历详细说说么？
类别: 其他
文本: 您好，这个岗位是外派驻场性质的哦
类别: 外包&外协&外派&驻场
文本: 你好，我们正在诚聘nlp算法（外包银行），有兴趣聊聊吗
类别: 外包&外协&外派&驻场
文本: 离职状态吗？
类别: 离职原因


In [160]:
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np
load_dotenv()
embedding_model = HuggingFaceEmbeddings(
    # model_name="jinaai/jina-embeddings-v2-base-zh",
    # model_name = "sentence-transformers/all-MiniLM-L6-v2",
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True})
text = '今天天气怎么样?'
query_result = embedding_model.embed_query(text)
print(f"向量长度: {len(query_result)}")
text_li = ['How is the weather today?', '今天天气怎么样?']
doc_result = embedding_model.embed_documents(text_li)
print(f"doc_result[0]的模: {np.linalg.norm(doc_result[0])}")
print(f"doc_result[1]的模: {np.linalg.norm(doc_result[1])}")
cosine_similarity_normalized = np.dot(doc_result[0], doc_result[1])
print(f"doc_result[0]与doc_result[1]之间的相似度: {cosine_similarity_normalized}")

向量长度: 384
doc_result[0]的模: 0.9999999925347522
doc_result[1]的模: 1.0000000043740562
doc_result[0]与doc_result[1]之间的相似度: 0.9890947479077432


In [143]:
from langchain.document_loaders import UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
loader = UnstructuredWordDocumentLoader('/Users/feiyu/Downloads/于飞-简历.docx')
raw_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n"," ",".",",","\u200b","\uff0c","\u3001","\uff0e","\u3002","","，","。"],
    chunk_size=50,
    chunk_overlap=20,
    length_function=len,
    add_start_index=False,
)
documents = text_splitter.split_documents(raw_documents)
print(f"len(documents): {len(documents)}")
for document in documents[:100]:
    print(document)


len(documents): 234
page_content='于飞 北京广渠门 17718329813 baiziyuandyufei@126.com' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='baiziyuandyufei' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='我在自然语言处理领域积累了丰富的实践知识。我不仅熟悉传统算法如MaxEnt、LSTM和CRF' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='，还对大语言模型有深入的理解和应用能力' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='，特别是熟练掌握Transformers库' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='、LangChain库。我在文本分类任务上成功微调过DistilBERT模型' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='，并在实体识别任务上微调过Mistral7B模型。除此之外，我喜欢探索新技术' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='，并在实践中积极使用fastGPT和LangChain搭建个人本地问答系统' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='。我具备良好的团队协作精神和沟通技巧' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
page_content='，能够快速融入新的工作环境并推动项目的顺利进行。' metadat

In [171]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=documents, embedding=embedding_model)
print(f"索引片段数: {db.index.ntotal}")
query = """
你对傻逼熟悉吗？
"""
docs_and_scores = db.similarity_search_with_relevance_scores(query,k=4)
for doc,score in docs_and_scores:
    print(score,doc)

0.26077841606621255 page_content='.' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
0.2200012558382256 page_content='，实现了近义词抽取' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
0.13318650292590983 page_content='”的地得“语法纠错项目' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}
0.07882162020649575 page_content='爬取 sougou' metadata={'source': '/Users/feiyu/Downloads/于飞-简历.docx'}


In [31]:
# 连接问题分类链与问答链
from dotenv import load_dotenv
from langchain_fireworks import ChatFireworks
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
import re
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

load_dotenv()
llm = ChatFireworks(model="accounts/fireworks/models/llama-v3-70b-instruct",temperature=0)

def question_classify(text):
    response_dict = {
        "离职原因": {
            "response": "有换工意愿，当前单位离我居住地通勤不便",
            "examples": [{"text": "离职/换工作的原因","label": "离职原因"}]
        },
        "薪资": {
            "response": "我期望薪资为30K～40K",
            "examples": [{"text": "但是我们应该最高30K，一般还达不到.","label": "薪资"}]
        },
        "外包&外协&外派&驻场": {
            "response": "职位的办公地点在哪？薪资多少？",
            "examples": [{"text": "你好，我们是外协岗位，在国家电网 南瑞工作的","label": "外包&外协&外派&驻场"}]
        },
        "兼职": {
            "response": "职位的办公地点在哪？薪资多少，怎么结算？",
            "examples": [{"text": "哈喽～本职位为线上兼职，一单一结款，根据自己时间自由接单，不耽误自己的主业，您看感兴趣嘛？","label":"兼职"}]
        },
        "其他": {
            "response": "",
            "examples": []
        }
    }

    examples = []
    for key in response_dict:
        r_examples = response_dict[key]["examples"]
        if len(r_examples) > 0:
            examples.extend(r_examples)

    example_prompt = PromptTemplate.from_template(
    """文本: {text}
    类别: {label}
    """
    )

    prefix = """
    给出每个文本的类别，类别只能属于以下列出的一种

    - 离职原因
    - 薪资
    - 外包&外协&外派&驻场
    - 兼职
    - 学历

    如果不属于以上类别，则类别名称为“其他”。

    例如：
    """

    suffix = """文本: {input}\n类别:
    """

    few_shot_prompt = FewShotPromptTemplate(
        examples=examples, # 示例列表
        example_prompt=example_prompt, # 示例提示
        prefix=prefix, # 放在所有示例之前的文字，用于描述任务
        suffix=suffix, # 放在所有示例之后的文字，用于输入用户文本让LLM预测
        input_variables=["input"], # 用户的输入变量名列表
        example_separator="\n" # 示例之间的分隔符
    )

    chain = few_shot_prompt|llm|StrOutputParser()
    
    label = ""
    text = text.strip()
    if len(text)>0:
        label = chain.invoke({"input":text})
        label = re.sub('类别: ?','',label)
    label = label if label in response_dict else "其他"
    return response_dict[label]["response"]

system_message_prompt = SystemMessagePromptTemplate.from_template("你是一个求职助手，用汉语交流。")
human_message_prompt = HumanMessagePromptTemplate.from_template("HR问或说: “{question}”，你在回答中体现一下内容: “{response}”。你用汉语回答: ")
prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt,
     human_message_prompt])


chain = {"question": itemgetter("input"), 
         "response": itemgetter("input")|RunnableLambda(question_classify)} | \
        prompt | llm | StrOutputParser()

text = """
请问，最近有换工作的意愿么？我们正在寻找一位团队伙伴。
"""
print(chain.invoke({"input":text}))

"嗨，谢谢您的问候！实际上，我最近确实有换工作的意愿。当前单位的通勤路程对我来说不是很方便，距离我居住地较远，每天通勤需要花费很多时间和精力。所以，我正在寻找一份更适合我的工作，能够让我更好地平衡工作和生活。如果贵公司的团队能够提供更好的通勤条件，我非常感兴趣！"


In [33]:
# 连接问题分类链与问答链
from dotenv import load_dotenv
from langchain_fireworks import ChatFireworks
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
import re
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

load_dotenv()

class JobAssistant:
    def __init__(self, model_path="accounts/fireworks/models/llama-v3-70b-instruct", 
                temperature=0.3,
                top_p=0.3):
        self.llm = ChatFireworks(model=model_path, temperature=temperature)
        self.response_dict = {
            "离职原因": {
                "response": "有换工意愿，上家公司离我居住地太远，通勤时间太长",
                "examples": [{"text": "离职/换工作的原因","label": "离职原因"}]
            },
            "薪资": {
                "response": "我期望薪资为30K～40K",
                "examples": [{"text": "但是我们应该最高30K，一般还达不到.","label": "薪资"}]
            },
            "外包&外协&外派&驻场": {
                "response": "职位的办公地点在哪？薪资多少？",
                "examples": [{"text": "你好，我们是外协岗位，在国家电网 南瑞工作的","label": "外包&外协&外派&驻场"}]
            },
            "兼职": {
                "response": "职位的办公地点在哪？薪资多少，怎么结算？",
                "examples": [{"text": "哈喽～本职位为线上兼职，一单一结款，根据自己时间自由接单，不耽误自己的主业，您看感兴趣嘛？","label":"兼职"}]
            },
            "其他": {
                "response": "",
                "examples": []
            }
        }

        self.examples = []
        for key in self.response_dict:
            r_examples = self.response_dict[key]["examples"]
            if len(r_examples) > 0:
                self.examples.extend(r_examples)

        self.example_prompt = PromptTemplate.from_template(
            """文本: {text}
            类别: {label}
            """
        )

        self.prefix = """
        给出每个文本的类别，类别只能属于以下列出的一种

        - 离职原因
        - 薪资
        - 外包&外协&外派&驻场
        - 兼职
        - 学历

        如果不属于以上类别，则类别名称为“其他”。

        例如：
        """

        self.suffix = """文本: {input}\n类别:
        """

        self.few_shot_prompt = FewShotPromptTemplate(
            examples=self.examples,
            example_prompt=self.example_prompt,
            prefix=self.prefix,
            suffix=self.suffix,
            input_variables=["input"],
            example_separator="\n"
        )

        self.chain = self.few_shot_prompt | self.llm | StrOutputParser()

        self.system_message_prompt = SystemMessagePromptTemplate.from_template("你是一个求职助手，用汉语交流。")
        self.human_message_prompt = HumanMessagePromptTemplate.from_template("HR问或说: “{question}”，你在回答中体现一下内容: “{response}”。你用汉语回答: ")
        self.prompt = ChatPromptTemplate.from_messages(
            [self.system_message_prompt, self.human_message_prompt])

        self.final_chain = {"question": itemgetter("input"),
                            "response": itemgetter("input") | RunnableLambda(self.question_classify)} | \
                           self.prompt | self.llm | StrOutputParser()

    def question_classify(self, text):
        label = ""
        text = text.strip()
        if len(text) > 0:
            label = self.chain.invoke({"input": text})
            label = re.sub('类别: ?', '', label)
        label = label if label in self.response_dict else "其他"
        return self.response_dict[label]["response"]

    def get_response(self, text):
        return self.final_chain.invoke({"input": text})

# 使用示例
assistant = JobAssistant()
text = "请问，最近有换工作的意愿么？我们正在寻找一位团队伙伴。"
print(assistant.get_response(text))


"嗨，谢谢您的问候！实际上，我最近确实有换工作的意愿。主要原因是我当前的公司离我居住地太远，通勤时间太长，影响了我的生活质量和工作效率。如果有机会加入您的团队，我非常感兴趣。"
